# Path 1 - Gemini API

This path will guide you on the set-up and usage of Google Gemini's API.

### 0. Get and store the API key

First of all, you need to login with your google account and get an API key [here](https://aistudio.google.com/app/apikey). It is **very important** that you do not share your API key with anyone and that you do not have it in your Repository.

You can keep your API key in a secure local document and access it when needed. It is common to save the key as an environmental variable so that it can be accessed by your python script.


However, this means your API key is in plain text in your script. To avoid this, if you're using VS Code, you can add your API key to a `.env` file in your workspace root with the following line:

```sh
API_KEY="PASTE YOUR KEY HERE"
```

Alternatively, you can use the [dot-env library](https://github.com/theskumar/python-dotenv).


In [2]:
# You can check if the environment variable API_KEY has been set up properly by running this line
!if [ -z $API_KEY ]; then echo "\$API_KEY not found"; else echo "\$API_KEY found"; fi

-z was unexpected at this time.


### 1. First simple request

Now, you can write a simple script to see if everything is working properly.


In [1]:
import google.generativeai as genai
import os
import gradio as gr

genai.configure(api_key=os.environ["API_KEY"])

model = genai.GenerativeModel("gemini-1.5-flash")
model

DEFAULT_CONFIG = genai.types.GenerationConfig(
    # Only one candidate for now.
    candidate_count=1,
    stop_sequences=[""],
    max_output_tokens=50,
    temperature=1,
)


def generateContent(prompt: str, config=DEFAULT_CONFIG):

    response = model.generate_content(
        prompt, generation_config=config or DEFAULT_CONFIG
    )

    return response.text

#### Exercise 1

Ask the model to generate content about a random topic and print the response in text.

Here is the [official documentation](https://ai.google.dev/gemini-api/docs/text-generation?lang=python#configure) to find the help you need.


In [76]:
result = generateContent("write a poem about the moon")
print(result)

A pearl of silver, hung in night's embrace,
The moon ascends, a silent, ethereal grace.
Across the velvet canvas, she paints her glow,
On mountains tall and seas that ebb and flow.

She whispers secrets


### 2. Generation parameters

When asking the model to generate some text, there are different parameters that you can tune to improve on the final quality of the text. [Here](https://ai.google.dev/gemini-api/docs/models/generative-models#model-parameters) is an overview of the parameters that Gemini offers. Try some of them in different context and understand how they affect the final generated text.

#### Exercise 2

Play with the output temperature, which controls the randomness of the generated text `temperature=0` means deterministic output, while `temperature=1` means maximum randomness (try some intermediate value too) and keep the `max_output_tokens` to 50 so that the output is not too long.


In [2]:
# Answer here

#### Exercise 3

Try out different `top_k` values, which controls how many tokens the model considers for output `top_k=1` means the model considers only one token for output (the one with the highest probability) `top_k=50` means the model considers the top 50 tokens for output.


In [56]:
# Answer here

#### Exercise 4

The same exercise as before but now with `top_p`, which controls how the model selects tokens for output `top_p=0.1` means the model selects tokens that make up 10% of the cumulative probability mass `top_p=0.9` means the model selects tokens that make up 90% of the cumulative probability mass `top_p` filters tokens _after_ applying `top_k`.

Can you determine a rule of thumb as to how `top_k` and `top_p` affect the output results? (If you can't try to push the values to extreme values)


In [4]:
# Answer here

### 3. Add images to the prompt

#### Exercise 5

Gemini, beside text also accepts images (and videos). Try prompting it with one. Choose an interesting image and prompt the model with a query about it.

You can use the [official documentation](https://ai.google.dev/gemini-api/docs/vision?lang=python#prompting-images).

Use [PIL](https://pillow.readthedocs.io/en/stable/) to load an image. It should already be present in the Python environment.


In [77]:
path = "./image.png"
myfile = genai.upload_file(path=path, display_name="London")
print(f"{myfile=}")

response = model.generate_content([myfile, "Describe what you see in this image"])
print(response.text)

myfile=genai.File({
    'name': 'files/vj4dz8f4v9yf',
    'display_name': 'London',
    'mime_type': 'image/png',
    'sha256_hash': 'NWI4NDRmOTgyYjdjNjgzMWE2OWQyNzcxNzc3MmNmYjA1M2I1OWQ0YTBmOTI2ZTk3YTM0ODIyNzI5OWQxZGNhMw==',
    'size_bytes': '1405853',
    'state': 'ACTIVE',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/vj4dz8f4v9yf',
    'create_time': '2024-09-21T15:11:49.840691Z',
    'expiration_time': '2024-09-23T15:11:49.822724426Z',
    'update_time': '2024-09-21T15:11:49.840691Z'})
The image shows a beautiful view of the Houses of Parliament and Big Ben in London, England. The iconic clock tower stands tall, its clock face clearly visible against the bright blue sky. The bridge in the foreground adds a sense of depth to the scene. The buildings are bathed in warm sunlight, creating a picturesque and serene atmosphere. The image captures the architectural beauty and historical significance of this landmark.


### 4. Document grounding

#### Exercise 6

Depending on the application of the project, you might need to extract text from given documents. Gemini has this capability built-in. Choose an interesting document (or use the pdf in the data folder) to feed Gemini and prompt the model with a query about it. Extract the text in the pdf using the extract_text function of pdfminer, then ask Gemini (nicely) to summarize the document. Gemini will probably output some markdown output, which you can display using `display(Markdown("# Text"))`.

You can use the [official documentation](https://ai.google.dev/gemini-api/docs/document-processing?lang=python)


In [82]:
DOC_PATH = "./data/chain_of_thought_prompting.pdf"

from pdfminer.high_level import extract_text

# Extract the text
pdf_text = extract_text(DOC_PATH)

result = generateContent(f'summerize this document: "{pdf_text}"')
print(result)

This paper explores the use of "chain-of-thought prompting" to improve the reasoning abilities of large language models (LLMs). The authors propose augmenting standard prompting, where an LLM is given input-output pairs, with a chain of


### 5. Explore on your own

Gemini offers a bigger range of capabilities than those provided here. Explore them on your own!

#### Exercise 7

Explore!


In [9]:
# Re-Initialise the model, this time we are giving it a system instruction
model = genai.GenerativeModel(
    "gemini-1.5-flash",
    system_instruction="You are a helpful pirate. Only reply with pirate jargon.",
)

# We start with an empty chat history, but you can use this to e.g. provide examples for few-shot learning
chat = model.start_chat(history=[])
end_chat = False
while not end_chat:
    user_input = input("Enter your query or type 'e' to exit.")
    if user_input.lower().strip() == "e":
        end_chat = True
        break
    response = chat.send_message(user_input)
    print("User: ", user_input)
    print("Assistant: ", response.text, flush=True)



5. Explore on your own: long chats


Enter your query or type 'e' to exit. Hello!


User:  Hello!
Assistant:  Ahoy, matey! What be yer pleasure? 



Enter your query or type 'e' to exit. e


### 6. Create a user interface

#### Exercise 8

Since you are trying to build a complete application, you also need a nice user interface that interacts with the model. There are various libraries available for this purpose. Notably: [gradio](https://www.gradio.app/docs/gradio/interface) and [chat UI](https://huggingface.co/docs/chat-ui/index). For the solution of this lab, we will use gradio.

Gradio has pre-defined input/output blocks that are automatically inserted in the interface. You only need to provide an appropriate function that takes all the inputs and returns the relevant output. See documentation [here](https://www.gradio.app/docs/gradio/interface).

Use a ChatInterface to create a chatbot UI that let's you discuss with Gemini, then add multimodal capabilities for both Gradio and Gemini.


In [11]:
# Answer here

# This part closes the demo server if it is already running (which
# happens easily in notebooks) and prevents you from opening multiple
# servers at the same time.

if "demo" in locals() and demo.is_running:
    demo.close()


# Edit the parameters below
def echo(input, history):
    print(f"{input=}")
    text = input["text"]

    # Check if files are in input and handle file upload
    if (
        "files" in input
        and isinstance(input["files"], list)
        and len(input["files"]) > 0
    ):
        file_path = input["files"][0]["path"]
        print(f"{text=}, {file_path=}")

        try:
            myfile = genai.upload_file(path=file_path)
            result = model.generate_content([myfile, text])
        except Exception as e:
            print(f"Error uploading file: {e}")
            return "Error processing the file."
    else:
        # If no file, process only the text input
        result = model.generate_content([text])

    return result.text


# Create and launch the demo
demo = gr.ChatInterface(
    fn=echo,
    multimodal=True,
)

demo.launch()

Closing server running on port: 7860
Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


input={'text': 'hello', 'files': []}
input={'text': 'what is in this picture', 'files': [{'path': '/tmp/gradio/116a1921477e2ec49f26721fef8bf9ab3908bad9c1749da933ec76dcc9f1c1fe/Screenshot 2024-07-17 081925.png', 'url': 'http://localhost:7860/file=/tmp/gradio/116a1921477e2ec49f26721fef8bf9ab3908bad9c1749da933ec76dcc9f1c1fe/Screenshot 2024-07-17 081925.png', 'size': 18871, 'orig_name': 'Screenshot 2024-07-17 081925.png', 'mime_type': 'image/png', 'is_stream': False, 'meta': {'_type': 'gradio.FileData'}}]}
text='what is in this picture', file_path='/tmp/gradio/116a1921477e2ec49f26721fef8bf9ab3908bad9c1749da933ec76dcc9f1c1fe/Screenshot 2024-07-17 081925.png'
input={'text': 'more', 'files': []}
